In [ ]:
# -*- coding: utf-8 -*-
# RAG (ChromaDB + SentenceTransformers) - No LangChain, No OpenAI
from __future__ import annotations
from pathlib import Path
import pandas as pd
import numpy as np
import re, os
from tqdm import tqdm
import chromadb
from sentence_transformers import SentenceTransformer

# =========================
# 설정
# =========================
CSV_PATHS = [
    "./data/경기 (전부완료).csv",
    "./data/광주 (전부완료).csv",
    "./data/제주 (전부완료).csv",
    # 필요시 더 추가 or glob로 수집
    # *중요*: 실제 경로/파일명에 맞게 수정
]
PERSIST_DIR = "./chroma_travel_db"         # ChromaDB 저장 폴더 (여기에 CSV 복사 X)
COLLECTION_NAME = "travel_places"
EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"  # 인덱싱/질의 동일 모델 사용
TOP_K = 5
DEFAULT_REGION_FILTER: str | None = None   # 예: "제주 (전부완료)" 로 region 제한, 아니면 None

# 문서 생성에 사용할 후보 컬럼들
WANTED_COLS = [
    "place","info","주소","이용시간","휴일","입장료","주차","화장실",
    "체험","체험가능 연령","이용가능시설","문의 및 안내","홈페이지"
]

# =========================
# 유틸
# =========================
def read_csv_safely(p: Path) -> pd.DataFrame:
    try:
        return pd.read_csv(p, encoding="utf-8")
    except UnicodeDecodeError:
        return pd.read_csv(p, encoding="cp949")

def clean_space(s: str) -> str:
    return re.sub(r"\s+", " ", str(s)).strip()

def slug(s: str) -> str:
    s = re.sub(r"\s+", "_", s.strip())
    s = re.sub(r"[^0-9A-Za-z가-힣_\-]", "", s)
    return s

def row_to_text(row: pd.Series, exist_cols: list[str]) -> str:
    """
    한 장소(row) -> 하나의 '문서 문자열'
    - place는 제목처럼 가장 앞
    - 존재하는 컬럼만 '컬럼명: 값' 형태로 연결
    """
    parts = []
    place = clean_space(row.get("place", ""))
    if place:
        parts.append(place)
    for c in exist_cols:
        if c == "place": 
            continue
        v = str(row.get(c, "")).strip()
        if v and v.lower() != "nan":
            parts.append(f"{c}: {clean_space(v)}")
    return " / ".join(parts)

# =========================
# 임베딩 준비
# =========================
_embedder = SentenceTransformer(EMBED_MODEL)
def embed_texts(texts: list[str]) -> list[list[float]]:
    # normalize_embeddings=True 로 코사인 유사도와 상성 ↑
    return _embedder.encode(texts, normalize_embeddings=True).tolist()

# =========================
# ChromaDB 초기화
# =========================
persist_dir = Path(PERSIST_DIR)
persist_dir.mkdir(parents=True, exist_ok=True)
client = chromadb.PersistentClient(path=str(persist_dir))

collection = client.get_or_create_collection(
    name=COLLECTION_NAME,
    metadata={"hnsw:space": "cosine"}  # 코사인 유사도
)

# =========================
# 1) 여러 CSV 인덱싱(업서트)
# =========================
def ingest_csvs(paths: list[str | Path], batch: int = 256) -> int:
    total = 0
    for p in paths:
        p = Path(p)
        if not p.exists():
            print(f"[경고] 파일 없음: {p}")
            continue

        try:
            df = read_csv_safely(p)
        except Exception as e:
            print(f"[skip] 읽기 실패 {p}: {e}")
            continue

        # 잡열(예: Unnamed: 0) 제거
        df = df.loc[:, ~df.columns.str.contains(r"^Unnamed")]
        if "place" not in df.columns:
            print(f"[skip] {p.name}: 'place' 컬럼 없음")
            continue

        df = df.dropna(subset=["place"]).drop_duplicates(subset=["place"])
        exist_cols = [c for c in WANTED_COLS if c in df.columns]
        region = p.stem  # 파일명(확장자 제거)을 메타 region에 저장

        docs, metas, ids = [], [], []
        for _, r in df.iterrows():
            place = clean_space(r["place"])
            text = row_to_text(r, exist_cols)
            if not text:
                continue
            doc_id = f"{region}::{slug(place)}"  # 파일명+장소로 고유화
            docs.append(text)
            metas.append({
                "place": place,
                "region": region,
                "source": p.name,
                "address": clean_space(r.get("주소","")),
            })
            ids.append(doc_id)

        # 배치 임베딩 & 업서트
        for i in tqdm(range(0, len(docs), batch), desc=f"Ingest {region}"):
            chunk_docs = docs[i:i+batch]
            chunk_ids  = ids[i:i+batch]
            chunk_meta = metas[i:i+batch]
            vecs = embed_texts(chunk_docs)
            collection.upsert(documents=chunk_docs, embeddings=vecs, metadatas=chunk_meta, ids=chunk_ids)

        total += len(ids)

    print(f"[완료] 인덱싱 문서 수: {total}")
    return total

# =========================
# 2) 검색 (질의 임베딩 → 유사도 검색)
# =========================
def retrieve(query: str, top_k: int = TOP_K, region: str | None = DEFAULT_REGION_FILTER):
    qv = embed_texts([query])[0]
    kwargs = dict(query_embeddings=[qv], n_results=top_k, include=["documents","metadatas","distances"])
    if region:
        kwargs["where"] = {"region": region}
    res = collection.query(**kwargs)
    docs = res["documents"][0] if res.get("documents") else []
    metas = res["metadatas"][0] if res.get("metadatas") else []
    dists = res["distances"][0] if res.get("distances") else []
    return list(zip(docs, metas, dists))

# =========================
# 3) LLM 없이 '추출적' 답변 만들기
#    - 규칙 기반 + 문장 임베딩 재랭킹으로 핵심 문장 골라 답변
# =========================
# 간단 문장 분할기(한국어/기호 기준)
# 문장 분할 (한글 종결형/영문 구두점/ " / " ) 대응
_SENT_DELIM = "§§"  # 문장 사이에만 잠깐 넣을 임시 구분자(본문에 나올 일 거의 없는 문자)

def split_sentences(text: str) -> list[str]:
    t = str(text)

    # 1) 한글 종결 + 마침표 패턴 뒤에 구분자 삽입
    #    예: "다.", "요.", "죠.", "함."
    t = re.sub(r"(다\.|요\.|죠\.|함\.)\s*", r"\1" + _SENT_DELIM, t)

    # 2) 일반 영문 구두점(. ! ?) 뒤에도 구분자 삽입
    t = re.sub(r"([.!?])\s*", r"\1" + _SENT_DELIM, t)

    # 3) 네가 문서 생성 시 사용한 " / " 도 문장 경계로 취급
    t = t.replace(" / ", _SENT_DELIM)

    # 4) 구분자로 split
    sents = [s.strip() for s in t.split(_SENT_DELIM) if s.strip()]

    # 5) 너무 짧은 토막 제거 (선택)
    sents = [s for s in sents if len(s) >= 4]
    return sents


def detect_intents(query: str) -> list[str]:
    q = query.lower()
    hits = []
    for field, kws in INTENT_KEYS.items():
        for kw in kws:
            if kw.lower() in q:
                hits.append(field)
                break
    return hits

def extract_field_lines(doc: str, desired_fields: list[str]) -> list[str]:
    """문서 문자열 내부에서 '필드명: 값' 형태 라인을 추출"""
    lines = []
    for field in desired_fields:
        # '필드명: ' 로 시작하는 구간만 골라내기 (문서 생성 규칙과 매칭)
        m = re.findall(fr"{re.escape(field)}:\s*([^/]+)", doc)
        for v in m:
            v = clean_space(v)
            if v:
                lines.append(f"{field}: {v}")
    return lines

def answer_extractive(question: str, top_k: int = TOP_K, region: str | None = DEFAULT_REGION_FILTER, n_sent: int = 4) -> str:
    # 1) 상위 문서 회수
    hits = retrieve(question, top_k=top_k, region=region)
    if not hits:
        return "관련 문서를 찾지 못했어요. 다른 키워드로 다시 물어봐 주세요."

    # 2) 질의 의도 파악(주소/입장료/이용시간/주차 등)
    fields = detect_intents(question)

    # 3) 선택지 A: 필드가 있다면 우선 해당 필드 라인을 긁어온 뒤, 부족하면 문장 랭킹
    picked_lines = []
    if fields:
        for doc, meta, _ in hits:
            picked_lines += extract_field_lines(doc, fields)
        picked_lines = list(dict.fromkeys(picked_lines))  # 중복 제거(순서 유지)

    # 4) 선택지 B: (보강) 문장 임베딩 기반 랭킹으로 핵심 문장 추가
    #    - 각 문서를 문장으로 쪼개고, 질의와 코사인 유사도 높은 순으로 n_sent개 뽑기
    if len(picked_lines) < n_sent:
        query_vec = np.array(embed_texts([question])[0])
        cand_sents = []
        sent_to_place = {}
        for doc, meta, _ in hits:
            place = meta.get("place","")
            sents = split_sentences(doc)
            if not sents: 
                continue
            vecs = np.array(embed_texts(sents))
            sims = (vecs @ query_vec)  # 이미 정규화돼 있으므로 내적=코사인
            top_idx = sims.argsort()[::-1][:max(2, n_sent)]  # 각 문서에서 상위 몇 개
            for i in top_idx:
                sent = clean_space(sents[i])
                if len(sent) >= 6:
                    cand_sents.append((sims[i], sent, place))
                    sent_to_place[sent] = place

        # 전체 상위 n_sent~2n_sent 정도로 압축
        cand_sents.sort(key=lambda x: x[0], reverse=True)
        for _, sent, place in cand_sents:
            if sent not in picked_lines:
                picked_lines.append(sent)
            if len(picked_lines) >= n_sent:
                break

    # 5) 답변 조립 (가이드 톤)
    #    - 첫 줄: 가장 관련 높은 place 요약
    top_places = [m.get("place","") for _, m, _ in hits if m.get("place")]
    top_places = [p for i,p in enumerate(top_places) if p and p not in top_places[:i]]
    header = ""
    if top_places:
        header = f"{top_places[0]} 관련 안내예요."

    # 6) 본문: 필드 라인/핵심 문장 3~5줄
    body_lines = picked_lines[:max(3, n_sent)]
    if not body_lines:
        # 그래도 비어있으면 info 필드만 추출
        for doc, _, _ in hits:
            m = re.findall(r"info:\s*([^/]+)", doc)
            if m:
                body_lines.append(clean_space(m[0]))
                if len(body_lines) >= 3:
                    break

    # 7) 출처 요약
    sources = ", ".join(top_places[:5])

    # 8) 최종 문자열
    parts = []
    if header: parts.append(header)
    parts += body_lines
    if sources:
        parts.append(f"(출처: {sources})")
    return "\n".join(parts)

# =========================
# 실행 예시
# =========================
if __name__ == "__main__":
    # 1) 최초 1회 인덱싱 (이미 했다면 주석 처리 가능)
    # ingest_csvs(CSV_PATHS)

    # 2) 검색 프리뷰
    # for doc, meta, dist in retrieve("경주 불국사 입장료 알려줘", region=None):
    #     print(f"[{meta.get('place')}] dist={dist:.4f}\n{doc[:180]}...\n")

    # 3) 과금 없이 '추출형' 답변
    # print(answer_extractive("경주 불국사 입장료 알려줘", region=None))
    # print(answer_extractive("제주 비 오는 날 가기 좋은 데이트 코스 추천해줘", region="제주 (전부완료)"))
    pass


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   8%|7         | 83.9M/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [11]:
# === (1) 새 모델 선택: e5/gte/bge 중 하나 ===
from sentence_transformers import SentenceTransformer

# 예) e5 권장
EMBED_MODEL = "intfloat/multilingual-e5-base"
# 예) gte 대안
# EMBED_MODEL = "Alibaba-NLP/gte-multilingual-base"
# 예) bge-m3 (가장 무겁고 성능↑, 1024차원)
# EMBED_MODEL = "BAAI/bge-m3"

_embedder = SentenceTransformer(EMBED_MODEL)

# === (2) e5/gte/bge 권장 프롬프트 규칙 ===
def embed_passages(texts):
    # 문서(패시지) 임베딩
    return _embedder.encode([f"passage: {t}" for t in texts], normalize_embeddings=True).tolist()

def embed_query(q: str):
    # 질의 임베딩
    return _embedder.encode([f"query: {q}"], normalize_embeddings=True).tolist()[0]

# (MiniLM 시절 함수가 남아 있어도 안전하게 우회)
def embed_texts(texts):
    # 과거 호출을 대비해 passage 규칙으로 라우팅
    return embed_passages(texts)

# === (3) 새 DB 폴더/컬렉션으로 분리 (차원 충돌 방지) ===
import chromadb
from pathlib import Path

PERSIST_DIR = "./chroma_travel_db_e5"      # ★ 새 폴더 이름
COLLECTION_NAME = "travel_places_e5"       # ★ 새 컬렉션 이름

Path(PERSIST_DIR).mkdir(parents=True, exist_ok=True)
_client = chromadb.PersistentClient(path=PERSIST_DIR)
collection = _client.get_or_create_collection(
    name=COLLECTION_NAME,
    metadata={"hnsw:space":"cosine"}
)

print("임베딩 모델:", EMBED_MODEL)

# === (4) ingest_csvs 내부 업서트 임베딩 교체 ===
# 네 노트북에 이미 존재하는 ingest_csvs가 있다면, 아래 함수를 재정의(오버라이드)합니다.
def ingest_csvs(paths: list[str | Path], batch: int = 256) -> int:
    import pandas as pd, re
    from tqdm import tqdm

    WANTED_COLS = [
        "place","info","주소","이용시간","휴일","입장료","주차","화장실",
        "체험","체험가능 연령","이용가능시설","문의 및 안내","홈페이지"
    ]

    def read_csv_safely(p: Path):
        try:
            return pd.read_csv(p, encoding="utf-8")
        except UnicodeDecodeError:
            return pd.read_csv(p, encoding="cp949")

    def clean_space(s: str) -> str:
        import re
        return re.sub(r"\s+", " ", str(s)).strip()

    def slug(s: str) -> str:
        import re
        s = re.sub(r"\s+", "_", s.strip())
        s = re.sub(r"[^0-9A-Za-z가-힣_\-]", "", s)
        return s

    def row_to_text(row, exist_cols):
        parts = []
        place = clean_space(row.get("place",""))
        if place:
            parts.append(place)
        for c in exist_cols:
            if c == "place": 
                continue
            v = str(row.get(c,"")).strip()
            if v and v.lower()!="nan":
                parts.append(f"{c}: {clean_space(v)}")
        return " / ".join(parts)

    total = 0
    for p in paths:
        p = Path(p)
        if not p.exists():
            print(f"[경고] 파일 없음: {p}")
            continue
        try:
            df = read_csv_safely(p)
        except Exception as e:
            print(f"[skip] 읽기 실패 {p}: {e}")
            continue

        df = df.loc[:, ~df.columns.str.contains(r"^Unnamed")]
        if "place" not in df.columns:
            print(f"[skip] {p.name}: 'place' 컬럼 없음")
            continue

        df = df.dropna(subset=["place"]).drop_duplicates(subset=["place"])
        exist_cols = [c for c in WANTED_COLS if c in df.columns]
        region = p.stem

        docs, metas, ids = [], [], []
        for _, r in df.iterrows():
            place = clean_space(r["place"])
            text = row_to_text(r, exist_cols)
            if not text: 
                continue
            doc_id = f"{region}::{slug(place)}"
            docs.append(text)
            metas.append({
                "place": place,
                "region": region,
                "source": p.name,
                "address": clean_space(r.get("주소","")),
            })
            ids.append(doc_id)

        for i in tqdm(range(0, len(docs), batch), desc=f"Ingest {region}"):
            chunk_docs = docs[i:i+batch]
            chunk_ids  = ids[i:i+batch]
            chunk_meta = metas[i:i+batch]
            vecs = embed_passages(chunk_docs)      # ★ 여기!
            collection.upsert(documents=chunk_docs, embeddings=vecs,
                              metadatas=chunk_meta, ids=chunk_ids)

        total += len(ids)

    print(f"[완료] 인덱싱 문서 수: {total}")
    return total

# === (5) retrieve 함수 내 질의 임베딩 교체 ===
TOP_K = 5
DEFAULT_REGION_FILTER = None  # 필요시 "제주 (전부완료)" 등으로 지정

def retrieve(query: str, top_k: int = TOP_K, region: str | None = DEFAULT_REGION_FILTER):
    qv = embed_query(query)  # ★ 여기!
    kwargs = dict(query_embeddings=[qv], n_results=top_k, include=["documents","metadatas","distances"])
    if region:
        kwargs["where"] = {"region": region}
    res = collection.query(**kwargs)
    docs = res["documents"][0] if res.get("documents") else []
    metas = res["metadatas"][0] if res.get("metadatas") else []
    dists = res["distances"][0] if res.get("distances") else []
    return list(zip(docs, metas, dists))

# === (6) 추천/추출 함수들도 새 임베딩으로 교체 ===
# split_sentences, extract_tags 등은 네가 이미 만든 그대로 사용

def recommend_places(
    query: str,
    region: str | None = None,
    k_docs: int = 20,
    top_n: int = 5,
    per_place_sent: int = 2,
    w_sim: float = 0.85,
    w_rule: float = 0.15,
):
    import numpy as np
    from collections import defaultdict

    hits = retrieve(query, top_k=k_docs, region=region)
    if not hits:
        return []

    qvec = np.array(embed_query(query))     # ★ 여기!

    desired = extract_tags(query)
    by_place = defaultdict(lambda: {"best_sim": -1.0, "sents": [], "tags": set(), "meta": None})
    for doc, meta, _dist in hits:
        place = meta.get("place", "")
        doc_tags = extract_tags(doc)
        sents = split_sentences(doc)
        if not sents:
            continue
        vecs = np.array(embed_passages(sents))  # ★ 여기!
        sims = (vecs @ qvec)
        top_idx = sims.argsort()[::-1][:max(1, per_place_sent)]
        top_sents = [sents[i] for i in top_idx]
        top_sim = float(sims[top_idx[0]])
        rule_bonus = len(doc_tags & desired)
        slot = by_place[place]
        if top_sim > slot["best_sim"]:
            slot["best_sim"] = top_sim
            slot["sents"] = top_sents
            slot["meta"] = meta
        slot["tags"] |= (doc_tags | desired)
        slot["rule_bonus"] = min(3, slot.get("rule_bonus", 0) + rule_bonus)

    results = []
    for place, slot in by_place.items():
        sim = (max(slot["best_sim"], 0.0) + 1.0) / 2.0
        rule = (slot.get("rule_bonus", 0)) / 3.0
        score = w_sim * sim + w_rule * rule
        results.append({
            "place": place,
            "score": float(score),
            "sentences": slot["sents"][:per_place_sent],
            "tags": sorted(list(slot["tags"])),
            "meta": slot["meta"],
        })
    results.sort(key=lambda x: x["score"], reverse=True)
    return results[:top_n]

def answer_extractive(question: str, top_k: int = TOP_K, region: str | None = DEFAULT_REGION_FILTER, n_sent: int = 4) -> str:
    import numpy as np, re
    hits = retrieve(question, top_k=top_k, region=region)
    if not hits:
        return "관련 문서를 찾지 못했어요. 다른 키워드로 다시 물어봐 주세요."

    fields = detect_intents(question)
    picked_lines = []
    if fields:
        for doc, meta, _ in hits:
            picked_lines += extract_field_lines(doc, fields)
        picked_lines = list(dict.fromkeys(picked_lines))

    if len(picked_lines) < n_sent:
        query_vec = np.array(embed_query(question))   # ★ 여기!
        cand_sents = []
        sent_to_place = {}
        for doc, meta, _ in hits:
            place = meta.get("place","")
            sents = split_sentences(doc)
            if not sents: 
                continue
            vecs = np.array(embed_passages(sents))    # ★ 여기!
            sims = (vecs @ query_vec)
            top_idx = sims.argsort()[::-1][:max(2, n_sent)]
            for i in top_idx:
                sent = sents[i].strip()
                if len(sent) >= 6:
                    cand_sents.append((sims[i], sent, place))
                    sent_to_place[sent] = place

        cand_sents.sort(key=lambda x: x[0], reverse=True)
        for _, sent, place in cand_sents:
            if sent not in picked_lines:
                picked_lines.append(sent)
            if len(picked_lines) >= n_sent:
                break

    top_places = [m.get("place","") for _, m, _ in hits if m.get("place")]
    top_places = [p for i,p in enumerate(top_places) if p and p not in top_places[:i]]
    header = f"{top_places[0]} 관련 안내예요." if top_places else ""
    sources = ", ".join(top_places[:5])

    parts = []
    if header: parts.append(header)
    parts += picked_lines[:max(3, n_sent)]
    if sources: parts.append(f"(출처: {sources})")
    return "\n".join(parts)

print("패치 적용 완료 ✅  — 새 컬렉션:", COLLECTION_NAME, "| DB:", PERSIST_DIR)


임베딩 모델: intfloat/multilingual-e5-base
패치 적용 완료 ✅  — 새 컬렉션: travel_places_e5 | DB: ./chroma_travel_db_e5


In [12]:
tmp = "여긴 산책로가 좋아요. 사진 찍기 좋고요. 주차는 가능 / 화장실 있음. 입장료는 무료예요."
print(split_sentences(tmp))
# ['여긴 산책로가 좋아요.', '사진 찍기 좋고요.', '주차는 가능', '화장실 있음.', '입장료는 무료예요.']


['여긴 산책로가 좋아요.', '사진 찍기 좋고요.', '주차는 가능', '화장실 있음.', '입장료는 무료예요.']


In [13]:
# ====== ① 태그 룰 (문서/질문에서 태깅) ======
import re, numpy as np
from collections import defaultdict

# 문서(text)와 질문(query)에서 공통으로 사용할 태깅 규칙 (간단 키워드 기반)
TAG_RULES = {
    "실내": r"실내|전시|박물관|미술관|갤러리|도서관|아쿠아리움|스파|온천",
    "카페": r"카페|티룸|로스터리|베이커리",
    "조용": r"조용|한적|정숙|휴식|힐링|고즈넉",
    "산책": r"산책|둘레길|숲길|정원|호수|하천|해안산책|트레일",
    "포토": r"사진|포토|전망|뷰|야경|인생샷|포토존",
    "체험": r"체험|프로그램|공방|만들기|시음|시연|투어",
    "아이": r"아기|유아|키즈|아이|어린이|유모차",
    "부모님": r"부모|어르신|노부모|효도|한옥|전통|천천히|완만",
    "연인": r"연인|데이트|감성|분위기",
    "반려동물": r"반려동물|펫|동물동반",
    "휠체어": r"휠체어|무장애|엘리베이터|경사로|장애인",
    "주차": r"주차|주차장",
    "비오는날": r"비\s*오는|우천|레인|실내",           # 우천 동선
    "여름": r"여름|계곡|폭포|그늘|숲|해수욕장|수영|냉",
    "겨울": r"겨울|온천|스파|따뜻|실내",
    "봄꽃": r"봄꽃|벚꽃|유채꽃|튤립|꽃",
    "단풍": r"단풍|가을",
    "액티비티": r"레저|액티비티|카약|서핑|패러글라이딩|승마|ATV|짚라인|클라이밍|집라인",
}

def extract_tags(text: str) -> set[str]:
    tags = set()
    for tag, pat in TAG_RULES.items():
        if re.search(pat, str(text), flags=re.IGNORECASE):
            tags.add(tag)
    return tags

# ====== ② 추천기: 질의 → 상위 문서 회수 → 문장 재랭킹 → 규칙 보너스 → 점수 ======
def recommend_places(
    query: str,
    region: str | None = None,     # 메타 region 필터 (예: "제주 (전부완료)")
    k_docs: int = 20,              # 먼저 가져올 문서 수
    top_n: int = 5,                # 최종 추천 개수
    per_place_sent: int = 2,       # 한 장소에서 이유로 보여줄 문장 수
    w_sim: float = 0.85,           # 임베딩 유사도 가중치
    w_rule: float = 0.15,          # 규칙/태그 보너스 가중치
):
    # 1) 상위 문서 회수
    hits = retrieve(query, top_k=k_docs, region=region)
    if not hits:
        return []

    # 2) 질의/문장 임베딩
    qvec = np.array(embed_texts([query])[0])

    # 3) 질문에서 희망 태그 추출
    desired = extract_tags(query)

    # 4) 장소별 스코어링
    by_place = defaultdict(lambda: {"best_sim": -1.0, "sents": [], "tags": set(), "meta": None})
    for doc, meta, _dist in hits:
        place = meta.get("place", "")
        doc_tags = extract_tags(doc)

        # 문장 단위 임베딩 & 유사도
        sents = split_sentences(doc)
        if not sents:
            continue
        vecs = np.array(embed_texts(sents))
        sims = (vecs @ qvec)  # 정규화되어 있으므로 내적=코사인

        # 이 문서에서 상위 문장 추출
        top_idx = sims.argsort()[::-1][:max(1, per_place_sent)]
        top_sents = [sents[i] for i in top_idx]
        top_sim = float(sims[top_idx[0]])

        # 규칙 보너스: (문서 태그 ∩ 질문 태그) 개수
        rule_bonus = len(doc_tags & desired)

        # 누적 업데이트(같은 place의 여러 문서가 들어왔을 때 최댓값 유지)
        slot = by_place[place]
        # 더 높은 유사도면 갈아끼우기
        if top_sim > slot["best_sim"]:
            slot["best_sim"] = top_sim
            slot["sents"] = top_sents
            slot["meta"] = meta
        # 태그는 합집합
        slot["tags"] |= (doc_tags | desired)
        # 보너스는 누적 최대 3 정도로 캡핑(과도한 편향 방지)
        slot["rule_bonus"] = min(3, slot.get("rule_bonus", 0) + rule_bonus)

    # 5) 최종 점수 = w_sim * 유사도 + w_rule * (보너스 정규화)
    results = []
    for place, slot in by_place.items():
        sim = max(slot["best_sim"], 0.0)                  # [-1,1] → [0,1] 보정
        sim = (sim + 1.0) / 2.0
        rule = (slot.get("rule_bonus", 0)) / 3.0          # 0~1
        score = w_sim * sim + w_rule * rule
        results.append({
            "place": place,
            "score": float(score),
            "sentences": slot["sents"][:per_place_sent],
            "tags": sorted(list(slot["tags"])),
            "meta": slot["meta"],
        })

    # 6) 점수순 정렬 & 상위 N개
    results.sort(key=lambda x: x["score"], reverse=True)

    # 7) (선택) 같은 place 중복 제거는 위에서 place key로 이미 해결됨
    return results[:top_n]

# ====== ③ 보기 좋게 출력 ======
def pretty_print_recos(recos: list[dict]):
    for i, r in enumerate(recos, 1):
        m = r["meta"] or {}
        addr = m.get("address", "")
        print(f"{i}. {r['place']}  |  score={r['score']:.3f}")
        if addr:
            print(f"   주소: {addr}")
        if r["sentences"]:
            for s in r["sentences"]:
                print(f"   • {s}")
        if r["tags"]:
            print(f"   태그: {', '.join(r['tags'])}")
        print()


In [15]:
ingest_csvs(CSV_PATHS)  # 실행 후 count 다시 확인


Ingest 제주 (전부완료): 100%|██████████| 4/4 [04:22<00:00, 65.64s/it]

[완료] 인덱싱 문서 수: 4257


4257

In [16]:
print("DB 폴더:", PERSIST_DIR)
print("컬렉션:", COLLECTION_NAME)
print("문서 수:", collection.count())  # 0이면 아직 안 들어감


DB 폴더: ./chroma_travel_db_e5
컬렉션: travel_places_e5
문서 수: 4257


In [17]:
# DB에서 몇 개 메타만 훑어서 region 값 확인
sample = collection.get(limit=20, include=["metadatas"])
regions = { (m or {}).get("region","") for m in sample.get("metadatas", []) }
print("존재하는 region들:", regions)


존재하는 region들: {'경기 (전부완료)'}


In [18]:
# 인덱싱할 때 저장된 region 문자열은 파일명 stem 그대로야.
from pathlib import Path
regions_expected = [Path(p).stem for p in CSV_PATHS]
print("예상 region들:", regions_expected)


예상 region들: ['경기 (전부완료)', '광주 (전부완료)', '제주 (전부완료)']


In [19]:
recos = recommend_places("비 오는 날 조용히 쉴 수 있는 실내 카페", region="제주 (전부완료)", top_n=5)
pretty_print_recos(recos)


1. 아침미소목장  |  score=0.905
   주소: 제주특별자치도 제주시 첨단동길 160-20 (월평동)
   • 화장실: 장애인 화장실 있음
   • 이용가능시설: 단체석 가능, 무선인터넷 사용가능
   태그: 겨울, 부모님, 비오는날, 실내, 아이, 조용, 주차, 체험, 카페, 휠체어

2. 헬로키티아일랜드  |  score=0.905
   주소: 제주특별자치도 서귀포시 안덕면 한창로 340
   • 화장실: 장애인 화장실 있음
   • 헬로키티 메모리 룸
   태그: 겨울, 비오는날, 실내, 아이, 조용, 주차, 체험, 카페, 휠체어

3. 점보빌리지  |  score=0.905
   주소: 제주특별자치도 서귀포시 안덕면 평화로319번길 31-11
   • 화장실: 장애인 전용 화장실 있음
   • 이곳은 코끼리들이 자유롭게 휴식하고 생활하는 공간으로, 코끼리와 사람들이 함께 공존할 수 있도록 조성된 곳이다.
   태그: 겨울, 비오는날, 실내, 조용, 주차, 체험, 카페, 휠체어

4. 송당 제주살롱  |  score=0.905
   주소: 제주특별자치도 제주시 구좌읍 송당2길 7-1
   • 북카페 내부에도 혼자 여행하는 이들이 편안하게 책을 읽을 수 있는 공간들을 배려한 느낌이다.
   • 간단한 음료와 함께 책을 읽을 수 있는 이곳은 북스테이를 할 수 있는 생각의 오름도 함께 운영한다.
   태그: 겨울, 비오는날, 실내, 조용, 주차, 체험, 카페

5. 휴림  |  score=0.902
   주소: 제주특별자치도 제주시 애월읍 광령남서길 40
   • 화장실: 있음
   • 주차: 가능
   태그: 겨울, 비오는날, 실내, 아이, 여름, 조용, 주차, 체험, 카페



In [10]:
recos = recommend_places("소개팅하기에 좋은 장소 추천해줘", region="경기 (전부완료)", top_n=5)
pretty_print_recos(recos)


InvalidArgumentError: Collection expecting embedding with dimension of 384, got 768